# Computer Organization Final Project - Mark Jordan and Brian Fedorka

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Probably gotta fix 2s complement on immediate values

In [ ]:
def Processor():
    # displays the purpose of program before starting the loop
    print('This program emulates the hardware of a processor.')
    print('The Program Counter (PC) will be displayed before each instruction is computed.')
    print('The outcome of the instruction will be shown after the computed instruction.')
    print(' ')

    ## filename = input('Enter the text file with the program: ', 's')

    # all matricies (instructions, memory, registers)
    ##Instruction_Memory = np.loadtxt(filename)

    i = 0
    Data_Memory_Array = []
    while i<=127:
        Data_Memory_Array.append(np.binary_repr(i, width=16))
        i+=1

    i=0
    Register_File = []
    while i<=15:
        Register_File.append(np.binary_repr(0, width=16))
        i+=1

    MAR = '0000000000000000' # Memory Access Register
    PC = 0 #starts the program counter at 0 (first instruction)
    
    # Main loop
    while PC <= 127:
        print("PC = ", PC) # shows instruction being performed

        # function call to retrieve current instruction. This returns the
        # current instruction from the Instruction_Memory matrix
        Instruction = Instruction_Fetch(PC,Instruction_Memory)

        # calls function to create control signals. It returns all signal values
        # along with the immediate (if it's R-type, the immediate is all 0's)
        Zero_Extend, Write_Choice, Opcode, Register_A, Register_B, Immediate, Jump, Jump_Amount, Branch, ALU_Operation, Mem_Read, Mem_Write, ALU_Src, Mem_toReg, Reg_Write = Control_Unit(Instruction)

        # function to sign extend the immediate (only for I-type)
        # is also done w/ J & R but has no effect on program outcome
        Sixteen_Bit_Immediate = Immediate_Extend(Zero_Extend, Immediate)

        # function that takes the value of each register and loads contents then
        # brings it back to the main file
        Data_A, Data_B, Shamt, RegisterA_Number = Register_Data_Fetch(Register_A, Register_B, Register_File, ALU_Operation)

        # Because the first register is always operated on(with exception of
        # Jump), it is just propegated through the bus. However, it is renamed
        # to ALU_Input1 for consistency
        ALU_Input1 = Data_A

        # function that takes RB and the Immediate and returns which
        # data will be operated on
        ALU_Input2 = ALU_MUX(Data_B, Sixteen_Bit_Immediate, ALU_Src)

        # function for the ALU. Performs all arithmetic operations. Active for
        # all instructions except jump, lw, and sw
        ALU_Output,Branch_Result = ALU(ALU_Input1, ALU_Input2, ALU_Operation, Shamt)

        # function that either writes data to a memory location or reads data in
        # from memory location
        Data_Memory_Array, Mem_Data = Data_Memory(Data_Memory_Array, MAR, Mem_Read, Mem_Write, ALU_Output)

        # function that acts as the memory multiplexor. If it's lw, the data is
        # from the data memory stage, otherwise, it's from the ALU
        Write_Data_Input = Memory_Mux(Mem_toReg, ALU_Output, Mem_Data)

        # file that writes data to the register files, Data is always written to
        # RA except when La operation is performed
        Register_File, MAR = Write_Data(Reg_Write, Write_Choice, Write_Data_Input, RegisterA_Number, Register_File, MAR)

        # adder that increments the Progam Counter by 1 each loop
        PC = Program_Counter_Increase(PC)

        # adder that performs the branch target. If the branch is not taken, the
        # value calculated has no effect of the program outcome
        PC_Branch, Branch_Amount = Branch_Adder(Branch, PC, Sixteen_Bit_Immediate)

        # MUX that determines an input signal that'll make branch address will be taken, or the
        # PC will stay the same (AND gate)
        Branch_Mux_Src = Branch_AND_Gate(Branch, Branch_Result)

        # either takes the branch or does nothing with PC (MUX for branching)
        PC = Branch_Mux(PC, PC_Branch, Branch_Mux_Src)

        # MUX that determines whether a jump is taken or not
        PC = Jump_Mux(PC, Jump, Jump_Amount)

        print(' ') #empty line after each instruction is run
        input("Press any key to continue ") #pause at the end of each instruction to display the result of the operation and where it is stored
    print(" ")
    print("The final state of each register is: ")
    print(" ")
    print("MAR = ", MAR)
    for i in range(0,16):
        print("Register ", i, " = ", Register_File[i])

In [ ]:
def Program_Counter_Increase(PC):
    PC += 1
    return PC

In [ ]:
def Instruction_Fetch(PC, Instruction_Memory):
    Temp_ins = Instruction_Memory[PC]
    Instruction = [Temp_ins[0:1],Temp_ins[1:2],Temp_ins[2:3],Temp_ins[3:4],Temp_ins[4:5],Temp_ins[5:6],Temp_ins[6:7],Temp_ins[7:8],Temp_ins[8:9],Temp_ins[9:10],Temp_ins[10:11],Temp_ins[11:12],Temp_ins[12:13],Temp_ins[13:14],Temp_ins[14:15],Temp_ins[15:16]]
    return Instruction

In [ ]:
def Control_Unit(Instruction):
    Opcode = Instruction[0:4]
    Zero_Extend = 0
    Jump_Amount = [0,0,0,0,0,0,0,0,0,0,0,0]
    Write_Choice = 0
    
    # R-Type Instructions
    if Opcode == [0,0,0,0]:
        Register_A = Instruction[4:8]
        Register_B = Instruction[8:12]
        Immediate = [0,0,0,0,0,0,0,0]
        Jump = 0
        Branch = 0
        Mem_Read = 0
        Mem_Write = 0
        ALU_Src = 0
        Mem_toReg = 0
        Reg_Write = 1
        Function_Code = Instruction[12:16]
        if Function_Code == [0,0,0,1]:
            print("R-type Instruction - Add")
            ALU_Operation = "Add"
        elif Function_Code == [0,0,1,0]:
            print("R-type Instruction - Subtract")
            ALU_Operation = "Sub"
        elif Function_Code == [0,0,1,1]:
            print("R-type Instruction - Nor")
            ALU_Operation = "Nor"
        elif Function_Code == [0,1,0,0]:
            print("R-type Instruction - Shift Logical Left")
            ALU_Operation = "SLL"
        elif Function_Code == [0,1,0,1]:
            print("R-type Instruction - Shift Logical Right")
            ALU_Operation = "SLR"
        else:
            print("Error, no matching Function_Code")
        print("Register adressing is used")
       
    
    
    # I-type Instructions
    if Opcode != [0,0,0,0] and Opcode != [0,1,1,1]:
        Immediate = Instruction[8:16]
        Register_A = Instruction[4:8]
        Register_B = [0,0,0,0] # unused
        Jump = 0
        
    # la instruction
    if Opcode == [0,0,0,1]:
        print("I-type Instruction - la")
        print("Base addressing is used")
        Write_Choice = 1
        Branch = 0
        Mem_Read = 0
        Mem_Write = 0
        ALU_Src = 1
        Mem_toReg = 0
        Reg_Write = 1
        ALU_Operation = "Add"
        
    # lw instruction
    if Opcode == [0,0,1,0]:
        print("I-type Instruction - lw ")
        print("Base addressing is used")
        Branch = 0
        Mem_Read = 1
        Mem_Write = 0
        ALU_Src = 0
        Mem_toReg = 1
        Reg_Write = 1
        ALU_Operation = "None"

    # sw instruction
    if Opcode == [0,0,1,1]:
        print("I-type Instruction - sw ")
        print("Base addressing is used")
        Branch = 0
        Mem_Read = 0
        Mem_Write = 1
        ALU_Src = 0
        Mem_toReg = 0
        Reg_Write = 0
        ALU_Operation = "None"
    
    # beq instructions
    if Opcode == [0,1,0,0]:
        print("I-type Instruction - beq ")
        print("PC-relative addressing is used if branch is taken")
        Branch = 1
        Mem_Read = 0
        Mem_Write = 0
        ALU_Src = 2      
        Mem_toReg = 0
        Reg_Write = 0
        ALU_Operation = "Add"
    
    # blt instructions
    if Opcode == [1,0,0,0]:
        print("I-type Instruction - blt ")
        print("PC-relative addressing is used if branch is taken")
        Branch = 1
        Mem_Read = 0
        Mem_Write = 0
        ALU_Src = 2      
        Mem_toReg = 0
        Reg_Write = 0
        ALU_Operation = "Sub"
        
    # addi instructions
    if Opcode == [0,1,0,1]:
        print("I-type Instruction - addi ")
        print("Immediate addressing is used")
        Branch = 0
        Mem_Read = 0
        Mem_Write = 0
        ALU_Src = 1
        Mem_toReg = 0
        Reg_Write = 1
        ALU_Operation = "Add"
    
    # ori instructions
    if Opcode == [0,1,1,0]:
        print("I-type Instruction - ori ")
        print("Immediate addressing is used")
        Zero_Extend = 1
        Branch = 0
        Mem_Read = 0
        Mem_Write = 0
        ALU_Src = 1
        Mem_toReg = 0
        Reg_Write = 1
        ALU_Operation = "OR"
        
    # Jump instructions
    if Opcode == [0,1,1,1]:
        print("J-type Instruction - Jump ")
        print("Pseudo-direct addressing is used")
        for i in range(4,16):
            Jump_Amount[i-4] = Instruction[i]
        Immediate = [0,0,0,0,0,0,0,0]
        Register_A = [0,0,0,0]
        Register_B = [0,0,0,0]
        Jump = 1
        Branch = 0
        Mem_Read = 0
        Mem_Write = 0
        ALU_Src = 0
        Mem_toReg = 0
        Reg_Write = 0
        ALU_Operation = "None"
        
    return Zero_Extend, Write_Choice, Opcode, Register_A, Register_B, Immediate, Jump, Jump_Amount, Branch, ALU_Operation, Mem_Read, Mem_Write, ALU_Src, Mem_toReg, Reg_Write

In [ ]:
def Immediate_Extend(Zero_Extend, Immediate):
    i = 0
    if Zero_Extend == 0: # sign extend
        while i <= 7:
            Immediate = np.insert(Immediate, 0, 0)
            i += 1
    if Zero_Extend == 1: # zero extend
        while i <= 7:
            a = Immediate[0]
            Immediate = np.insert(Immediate, 0, a)
            i += 1
    Sixteen_Bit_Immediate = Immediate
    return Sixteen_Bit_Immediate

In [ ]:
def Register_Data_Fetch(Register_A, Register_B, Register_File, ALU_Operation):
    a = Register_A[0]
    b = Register_A[1]
    c = Register_A[2]
    d = Register_A[3]
    RegisterA_Number = 8*a+4*b+2*c+d
   
    a = Register_B[0]
    b = Register_B[1]
    c = Register_B[2]
    d = Register_B[3]
    RegisterB_Number = 8*a+4*b+2*c+d
    
    Data_A = Register_File[RegisterA_Number]
    Data_B = Register_File[RegisterB_Number]
    
    if ALU_Operation == "SLL" or ALU_Operation == "SLR":
        Shamt = RegisterB_Number

    # RegisterA_Number is used for Write_Data to write to register A.
    return Data_A, Data_B, Shamt, RegisterA_Number

# ALU_Input2 is sixteen bit extended, or Data_B, or 16 0s. 
# ALU_Input1 is Data_A
they are all less than 7 bits, plus sign extension

In [ ]:
def ALU_Mux(Data_B, Sixteen_Bit_Immediate, ALU_Src):
    if ALU_Src == 0:
        ALU_Input2 = Data_B
    if ALU_Src == 1:
        ALU_Input2 = Sixteen_Bit_Immediate
    if ALU_Src == 2:
        ALU_Input2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    return ALU_Input2

In [ ]:
def ALU(ALU_Input1, ALU_Input2, ALU_Operation, Shamt):
    Branch_Result = 0
    ALU_Output = np.zeros_like(ALU_Input2)
    a = int(ALU_Input1[0])
    b = int(ALU_Input1[1])
    c = int(ALU_Input1[2])
    d = int(ALU_Input1[3])
    e = int(ALU_Input1[4])
    f = int(ALU_Input1[5])
    g = int(ALU_Input1[6])
    h = int(ALU_Input1[7])
    j = int(ALU_Input1[8])
    k = int(ALU_Input1[9])
    l = int(ALU_Input1[10])
    m = int(ALU_Input1[11])
    n = int(ALU_Input1[12])
    o = int(ALU_Input1[13])
    p = int(ALU_Input1[14])
    q = int(ALU_Input1[15])
    ALU_Input1_Number = q+2*p+4*o+8*n+16*m+32*l+64*k+128*j+256*h+512*g+1024*f+2048*e+4096*d+8192*c+16384*b+32768*a
    
    a = ALU_Input2[0]
    b = ALU_Input2[1]
    c = ALU_Input2[2]
    d = ALU_Input2[3]
    e = ALU_Input2[4]
    f = ALU_Input2[5]
    g = ALU_Input2[6]
    h = ALU_Input2[7]
    j = ALU_Input2[8]
    k = ALU_Input2[9]
    l = ALU_Input2[10]
    m = ALU_Input2[11]
    n = ALU_Input2[12]
    o = ALU_Input2[13]
    p = ALU_Input2[14]
    q = ALU_Input2[15]
    ALU_Input2_Number = q+2*p+4*o+8*n+16*m+32*l+64*k+128*j+256*h+512*g+1024*f+2048*e+4096*d+8192*c+16384*b+32768*a
    
    
    ## MAY HAVE ISSUES WITH ALU_OUTPUT FOR ADD AND SUB
    
    if ALU_Operation == "Add":
        ALU_Output_Number = ALU_Input1_Number+ALU_Input2_Number
        ALU_Output = np.binary_repr(ALU_Output_Number, width=16)
        if ALU_Output_Number == 0:
            Branch_Result = 1
        
    if ALU_Operation == "Sub":
        ALU_Output_Number = ALU_Input1_Number-ALU_Input2_Number
        ALU_Output = np.binary_repr(ALU_Output_Number, width=16)
        if ALU_Output_Number < 0:
            Branch_Result = 1
            
    if ALU_Operation == "Or":
        i = 0
        while i < 15:
            a = ALU_Input1[i]
            b = ALU_Input2[i]
            if a == 1 or b == 1:
                ALU_Output[i] = 1
            else:
                ALU_Output[i] = 0
            i += 1
        
    if ALU_Operation == "Nor":
        i = 0
        while i < 15:
            a = ALU_Input1[i]
            b = ALU_Input2[i]
            if a == 1 or b == 1:
                ALU_Output[i] = 0
            else:
                ALU_Output[i] = 1
            i += 1
            
    if ALU_Operation == "SLL":
        i = 0
        while i < Shamt:
            ALU_Input1.append(0)
            del ALU_Input1[0]
            i += 1
        ALU_Output = ALU_Input1
        
    if ALU_Operation == "SLR":
        i = 0
        while i < Shamt:
            ALU_Input1.insert(0,0)
            del ALU_Input1[-1]
            i += 1
        ALU_Output = ALU_Input1
    
    return ALU_Output, Branch_Result

In [ ]:
def Data_Memory(Data_Memory_Array, MAR, Mem_Read, Mem_Write, ALU_Output):
    MAR_Number = int(MAR, 2)
    if Mem_Write == 1:
        Data_Memory_Array[MAR_Number] = ALU_Output
        Mem_Data = Data_Memory_Array[MAR_Number] # just here to have an output
    if Mem_Read == 1:
        Mem_Data = Data_Memory_Array[MAR_Number]
    return Data_Memory_Array, Mem_Data

In [ ]:
def Memory_Mux(Mem_toReg, ALU_Output, Mem_Data):
    if Mem_toReg == 0:
        Write_Data_Input = ALU_Output
    if Mem_toReg == 1:
        Write_Data_Input = Mem_Data
    return Write_Data_Input

In [ ]:
def Write_Data(Reg_Write, Write_Choice, Write_Data_Input, RegisterA_Number, Register_File, MAR):
    if Reg_Write == 1:
        if Write_Choice == 0:
            Register_File[RegisterA_Number] = Write_Data_Input
        if Write_Choice == 1:
            MAR = Write_Data_Input
    return Register_File, MAR

In [ ]:
def Branch_Adder(Branch, PC, Sixteen_Bit_Immediate):
    Sixteen_Bit_Immediate_String = "".join(str(e) for e in Sixteen_Bit_Immediate)
    Sixteen_Bit_Immediate_Number = int(Sixteen_Bit_Immediate_String,2)
    Branch_Amount = Sixteen_Bit_Immediate_Number
    PC_Branch = PC+Branch_Amount    
    return PC_Branch, Branch_Amount

In [ ]:
def Branch_AND_Gate(Branch, Branch_Result):
    if Branch == 1:
        if Branch_Result == 1:
            Branch_Mux_Src = 1
        else:
            Branch_Mux_Src = 0
    else:
        Branch_Mux_Src = 0
    return Branch_Mux_Src

In [ ]:
def Branch_Mux(PC, PC_Branch, Branch_Mux_Src):
    if Branch_Mux_Src == 0:
        PC = PC
    if Branch_Mux_Src == 1:
        PC = PC_Branch
    return PC

In [ ]:
def Jump_Mux(PC, Jump, Jump_Amount):
    if Jump == 0:
        PC = PC
    if Jump == 1:
        PC = Jump_Amount
    return PC